In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

In [ ]:
import numpy as np
import pandas as pd

import PyPDF2
import camelot
import re

In [ ]:
def get_after_split(x):
    x_split = re.split("\n", x)
    if len(x_split) == 1:
        return x_split[0]
    else:
        return "".join(x_split[1:])

In [ ]:
def separate_artikel_deutsch(artikel_deutsch):
    artikel_deutsch_split = artikel_deutsch.split("\n")
    if len(artikel_deutsch_split) == 1:
        return ("", artikel_deutsch_split[0])
    else:
        return (artikel_deutsch_split[0], artikel_deutsch_split[1])

In [ ]:
def separate_plural_englisch(plural_englisch):
    plural_englisch_split = plural_englisch.split("\n")
    if len(plural_englisch_split) == 1:
        return ("", plural_englisch_split[0])
    else:
        return (plural_englisch_split[0], plural_englisch_split[1])

# wortschatzliste_einfach-gut_a1-1_englisch

In [ ]:
pdf_name = "data/raw/new_vocabulary/wortschatzliste_einfach-gut_a1-1_englisch.pdf"

## Get number of pages of pdf document

In [ ]:
# importing required modules
import PyPDF2

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(stream=pdf_name)

# printing number of pages in pdf file
num_pages = pdfReader.numPages

In [ ]:
print(num_pages)

## Get tables

In [ ]:
import camelot

In [ ]:
# Syntax of the camelot.read_pdf function
tables = camelot.read_pdf(pdf_name, pages="1-{}".format(num_pages), flavor="stream")

## Format all tables

In [ ]:
i = 15
table_page1 = tables[i].df
table_page1

In [ ]:
deutsch_englisch = pd.DataFrame()

In [ ]:
for i in range(num_pages):
    # i = 14
    print(i)

    table_page1 = tables[i].df

    # remove first row
    table_page1 = table_page1[1:]

    if i == 0:
        # 1st row is column names
        table_page1 = table_page1[1:]

    if i in [0, 1, 3]:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        # remove example
        del table_page1["beispiel"]
    if i in [9, 10]:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        table_page1["englisch"] = table_page1["englisch"].map(get_after_split)
        # remove example
        del table_page1["beispiel"]
    if i in [2, 4, 7, 8, 11, 12, 13, 14]:
        table_page1.columns = ["artikel", "deutsch_plural", "plural", "englisch"]
        # remove plural
        del table_page1["plural"]
    if i in [5, 6]:
        table_page1.columns = [
            "artikel",
            "deutsch_plural",
            "plural",
            "englisch",
            "beispiel",
        ]
        # remove example & plural
        del table_page1["beispiel"]
        del table_page1["plural"]

    # remove row when no word in deutsch and english
    table_page1 = table_page1[
        (table_page1["deutsch_plural"] != "") & (table_page1["englisch"] != "")
    ]

    # merge artikel and deutsch
    table_page1["deutsch_plural"] = table_page1[["artikel", "deutsch_plural"]].agg(
        " ".join, axis=1
    )

    # if article, add "the" to english
    table_page1["englisch"] = table_page1.apply(
        lambda x: ("the " + x["englisch"]) if (x["artikel"] != "") else x["englisch"],
        axis=1,
    )

    # remove artikel column
    del table_page1["artikel"]

    # if deutsch contains "(Sie", add "to" to englisch
    table_page1["is_verb"] = table_page1["deutsch_plural"].map(
        lambda x: ("(Sie" in x) or ("(ich" in x)
    )

    table_page1["englisch"] = table_page1.apply(
        lambda x: ("to " + x["englisch"])
        if (x["englisch"][:3] != "to ") & (x["is_verb"])
        else x["englisch"],
        axis=1,
    )

    # in deutsch or englisch, remove everything after the first "/" or "(" or "\n"
    table_page1["deutsch"] = table_page1["deutsch_plural"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )
    # in deutsch, remove everything after the first "/" or "(" or "\n"
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )

    # strip extra-space in word
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: re.sub("\s+", " ", x))
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.sub("\s+", " ", x)
    )

    # strip space on left and right
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: x.strip())
    table_page1["englisch"] = table_page1["englisch"].map(lambda x: x.strip())

    del table_page1["deutsch_plural"]
    del table_page1["is_verb"]

    # remove row when no word in deutsch or english
    table_page1 = table_page1[
        (table_page1["deutsch"] != "")
        & (table_page1["englisch"] != "")
        & (~table_page1["deutsch"].isna())
        & (~table_page1["englisch"].isna())
    ]

    deutsch_englisch = deutsch_englisch.append(table_page1[["deutsch", "englisch"]])

In [ ]:
deutsch_englisch

In [ ]:
deutsch_englisch.to_csv(
    "data/raw/new_vocabulary/wortschatzliste_einfach-gut_a1-1_englisch.csv", index=False
)

# wortschatzliste_einfach-gut_a1-2_englisch

In [ ]:
pdf_name = "data/raw/new_vocabulary/wortschatzliste_einfach-gut_a1-2_englisch.pdf"

## Get number of pages of pdf document

In [ ]:
# importing required modules
import PyPDF2

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(stream=pdf_name)

# printing number of pages in pdf file
num_pages = pdfReader.numPages

In [ ]:
print(num_pages)

## Get tables

In [ ]:
import camelot

In [ ]:
# Syntax of the camelot.read_pdf function
tables = camelot.read_pdf(pdf_name, pages="1-{}".format(num_pages), flavor="stream")

## Format all tables

In [ ]:
i = 13
table_page1 = tables[i].df
table_page1

In [ ]:
deutsch_englisch = pd.DataFrame()

In [ ]:
for i in range(num_pages):
    # i = 14
    print(i)

    table_page1 = tables[i].df

    # remove first row
    table_page1 = table_page1[1:]

    if i == 0:
        # 1st row is column names
        table_page1 = table_page1[1:]

    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        # remove example
        del table_page1["beispiel"]
    if i in [10]:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        table_page1["englisch"] = table_page1["englisch"].map(get_after_split)
        # remove example
        del table_page1["beispiel"]
    if i in [1, 2, 3, 4, 6, 7, 13]:
        table_page1.columns = ["artikel", "deutsch_plural", "plural", "englisch"]
        # remove plural
        del table_page1["plural"]
    if i in [0, 5, 8, 9, 11, 12]:
        table_page1.columns = [
            "artikel",
            "deutsch_plural",
            "plural",
            "englisch",
            "beispiel",
        ]
        # remove example & plural
        del table_page1["beispiel"]
        del table_page1["plural"]

    # remove row when no word in deutsch and english
    table_page1 = table_page1[
        (table_page1["deutsch_plural"] != "") & (table_page1["englisch"] != "")
    ]

    # merge artikel and deutsch
    table_page1["deutsch_plural"] = table_page1[["artikel", "deutsch_plural"]].agg(
        " ".join, axis=1
    )

    # if article, add "the" to english
    table_page1["englisch"] = table_page1.apply(
        lambda x: ("the " + x["englisch"]) if (x["artikel"] != "") else x["englisch"],
        axis=1,
    )

    # remove artikel column
    del table_page1["artikel"]

    # if deutsch contains "(Sie", add "to" to englisch
    table_page1["is_verb"] = table_page1["deutsch_plural"].map(
        lambda x: ("(Sie" in x) or ("(ich" in x)
    )

    table_page1["englisch"] = table_page1.apply(
        lambda x: ("to " + x["englisch"])
        if (x["englisch"][:3] != "to ") & (x["is_verb"])
        else x["englisch"],
        axis=1,
    )

    # in deutsch or englisch, remove everything after the first "/" or "(" or "\n"
    table_page1["deutsch"] = table_page1["deutsch_plural"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )
    # in deutsch, remove everything after the first "/" or "(" or "\n"
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )

    # strip extra-space in word
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: re.sub("\s+", " ", x))
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.sub("\s+", " ", x)
    )

    # strip space on left and right
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: x.strip())
    table_page1["englisch"] = table_page1["englisch"].map(lambda x: x.strip())

    del table_page1["deutsch_plural"]
    del table_page1["is_verb"]

    # remove row when no word in deutsch or english
    table_page1 = table_page1[
        (table_page1["deutsch"] != "")
        & (table_page1["englisch"] != "")
        & (~table_page1["deutsch"].isna())
        & (~table_page1["englisch"].isna())
    ]

    deutsch_englisch = deutsch_englisch.append(table_page1[["deutsch", "englisch"]])

In [ ]:
deutsch_englisch

In [ ]:
deutsch_englisch.to_csv(
    "data/raw/new_vocabulary/wortschatzliste_einfach-gut_a1-2_englisch.csv", index=False
)

# wortschatzliste_einfach-gut_a2-1_englisch

In [ ]:
pdf_name = "data/raw/new_vocabulary/wortschatzliste_einfach-gut_a2-1_englisch.pdf"

## Get number of pages of pdf document

In [ ]:
# importing required modules
import PyPDF2

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(stream=pdf_name)

# printing number of pages in pdf file
num_pages = pdfReader.numPages

In [ ]:
print(num_pages)

## Get tables

In [ ]:
import camelot

In [ ]:
# Syntax of the camelot.read_pdf function
tables = camelot.read_pdf(pdf_name, pages="1-{}".format(num_pages), flavor="stream")

## Format all tables

In [ ]:
i = 12
table_page1 = tables[i].df
table_page1

In [ ]:
deutsch_englisch = pd.DataFrame()

In [ ]:
for i in range(num_pages):
    # i = 14
    print(i)

    table_page1 = tables[i].df

    # remove first row
    table_page1 = table_page1[1:]

    if i == 0:
        # 1st row is column names
        table_page1 = table_page1[1:]

    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        # remove example
        del table_page1["beispiel"]
    if i in [0]:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        table_page1["englisch"] = table_page1["englisch"].map(get_after_split)
        # remove example
        del table_page1["beispiel"]
    if i in [1, 6, 7, 8, 9, 10]:
        table_page1.columns = ["artikel", "deutsch_plural", "plural", "englisch"]
        # remove plural
        del table_page1["plural"]
    if i in [3, 4]:
        table_page1.columns = [
            "artikel",
            "deutsch_plural",
            "plural",
            "englisch",
            "beispiel",
        ]
        # remove example & plural
        del table_page1["beispiel"]
        del table_page1["plural"]
    if i in [2, 11]:
        table_page1.columns = ["artikel_deutsch", "plural", "englisch"]
        table_page1["artikel"], table_page1["deutsch_plural"] = zip(
            *table_page1["artikel_deutsch"].map(separate_artikel_deutsch)
        )
        del table_page1["artikel_deutsch"]
        del table_page1["plural"]
    if i in [5]:
        table_page1.columns = ["artikel_deutsch", "plural", "englisch", "beispiel"]
        table_page1["artikel"], table_page1["deutsch_plural"] = zip(
            *table_page1["artikel_deutsch"].map(separate_artikel_deutsch)
        )
        del table_page1["artikel_deutsch"]
        del table_page1["plural"]
        del table_page1["beispiel"]

    # remove row when no word in deutsch and english
    table_page1 = table_page1[
        (table_page1["deutsch_plural"] != "") & (table_page1["englisch"] != "")
    ]

    # merge artikel and deutsch
    table_page1["deutsch_plural"] = table_page1[["artikel", "deutsch_plural"]].agg(
        " ".join, axis=1
    )

    # if article, add "the" to english
    table_page1["englisch"] = table_page1.apply(
        lambda x: ("the " + x["englisch"]) if (x["artikel"] != "") else x["englisch"],
        axis=1,
    )

    # remove artikel column
    del table_page1["artikel"]

    # if deutsch contains "(Sie", add "to" to englisch
    table_page1["is_verb"] = table_page1["deutsch_plural"].map(
        lambda x: ("(Sie" in x) or ("(ich" in x)
    )

    table_page1["englisch"] = table_page1.apply(
        lambda x: ("to " + x["englisch"])
        if (x["englisch"][:3] != "to ") & (x["is_verb"])
        else x["englisch"],
        axis=1,
    )

    # in deutsch or englisch, remove everything after the first "/" or "(" or "\n"
    table_page1["deutsch"] = table_page1["deutsch_plural"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )
    # in deutsch, remove everything after the first "/" or "(" or "\n"
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )

    # strip extra-space in word
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: re.sub("\s+", " ", x))
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.sub("\s+", " ", x)
    )

    # strip space on left and right
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: x.strip())
    table_page1["englisch"] = table_page1["englisch"].map(lambda x: x.strip())

    del table_page1["deutsch_plural"]
    del table_page1["is_verb"]

    # remove row when no word in deutsch or english
    table_page1 = table_page1[
        (table_page1["deutsch"] != "")
        & (table_page1["englisch"] != "")
        & (~table_page1["deutsch"].isna())
        & (~table_page1["englisch"].isna())
    ]

    deutsch_englisch = deutsch_englisch.append(table_page1[["deutsch", "englisch"]])

In [ ]:
deutsch_englisch

In [ ]:
deutsch_englisch.to_csv(
    "data/raw/new_vocabulary/wortschatzliste_einfach-gut_a2-1_englisch.csv", index=False
)

# wortschatzliste_einfach-gut_a2-2_englisch

In [ ]:
pdf_name = "data/raw/new_vocabulary/wortschatzliste_einfach-gut_a2-2_englisch.pdf"

## Get number of pages of pdf document

In [ ]:
# importing required modules
import PyPDF2

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(stream=pdf_name)

# printing number of pages in pdf file
num_pages = pdfReader.numPages

In [ ]:
print(num_pages)

## Get tables

In [ ]:
import camelot

In [ ]:
# Syntax of the camelot.read_pdf function
tables = camelot.read_pdf(pdf_name, pages="1-{}".format(num_pages), flavor="stream")

## Format all tables

In [ ]:
i = 14
table_page1 = tables[i].df
table_page1

In [ ]:
deutsch_englisch = pd.DataFrame()

In [ ]:
for i in range(num_pages):
    # i = 14
    print(i)

    table_page1 = tables[i].df

    # remove first row
    table_page1 = table_page1[1:]

    if i == 0:
        # 1st row is column names
        table_page1 = table_page1[1:]

    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        # remove example
        del table_page1["beispiel"]
    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        table_page1["englisch"] = table_page1["englisch"].map(get_after_split)
        # remove example
        del table_page1["beispiel"]
    if i in [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
        table_page1.columns = ["artikel", "deutsch_plural", "plural", "englisch"]
        # remove plural
        del table_page1["plural"]
    if i in [0]:
        table_page1.columns = [
            "artikel",
            "deutsch_plural",
            "plural",
            "englisch",
            "beispiel",
        ]
        # remove example & plural
        del table_page1["beispiel"]
        del table_page1["plural"]
    if i in [3]:
        table_page1.columns = ["artikel_deutsch", "plural", "englisch"]
        table_page1["artikel"], table_page1["deutsch_plural"] = zip(
            *table_page1["artikel_deutsch"].map(separate_artikel_deutsch)
        )
        del table_page1["artikel_deutsch"]
        del table_page1["plural"]
    if i in []:
        table_page1.columns = ["artikel_deutsch", "plural", "englisch", "beispiel"]
        table_page1["artikel"], table_page1["deutsch_plural"] = zip(
            *table_page1["artikel_deutsch"].map(separate_artikel_deutsch)
        )
        del table_page1["artikel_deutsch"]
        del table_page1["plural"]
        del table_page1["beispiel"]

    # remove row when no word in deutsch and english
    table_page1 = table_page1[
        (table_page1["deutsch_plural"] != "") & (table_page1["englisch"] != "")
    ]

    # merge artikel and deutsch
    table_page1["deutsch_plural"] = table_page1[["artikel", "deutsch_plural"]].agg(
        " ".join, axis=1
    )

    # if article, add "the" to english
    table_page1["englisch"] = table_page1.apply(
        lambda x: ("the " + x["englisch"]) if (x["artikel"] != "") else x["englisch"],
        axis=1,
    )

    # remove artikel column
    del table_page1["artikel"]

    # if deutsch contains "(Sie", add "to" to englisch
    table_page1["is_verb"] = table_page1["deutsch_plural"].map(
        lambda x: ("(Sie" in x) or ("(ich" in x)
    )

    table_page1["englisch"] = table_page1.apply(
        lambda x: ("to " + x["englisch"])
        if (x["englisch"][:3] != "to ") & (x["is_verb"])
        else x["englisch"],
        axis=1,
    )

    # in deutsch or englisch, remove everything after the first "/" or "(" or "\n"
    table_page1["deutsch"] = table_page1["deutsch_plural"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )
    # in deutsch, remove everything after the first "/" or "(" or "\n"
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )

    # strip extra-space in word
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: re.sub("\s+", " ", x))
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.sub("\s+", " ", x)
    )

    # strip space on left and right
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: x.strip())
    table_page1["englisch"] = table_page1["englisch"].map(lambda x: x.strip())

    del table_page1["deutsch_plural"]
    del table_page1["is_verb"]

    # remove row when no word in deutsch or english
    table_page1 = table_page1[
        (table_page1["deutsch"] != "")
        & (table_page1["englisch"] != "")
        & (~table_page1["deutsch"].isna())
        & (~table_page1["englisch"].isna())
    ]

    deutsch_englisch = deutsch_englisch.append(table_page1[["deutsch", "englisch"]])

In [ ]:
deutsch_englisch

In [ ]:
deutsch_englisch.to_csv(
    "data/raw/new_vocabulary/wortschatzliste_einfach-gut_a2-2_englisch.csv", index=False
)

# wortschatzliste_einfach-gut_b1-1_englisch

In [ ]:
pdf_name = "data/raw/new_vocabulary/wortschatzliste_einfach-gut_b1-1_englisch.pdf"

## Get number of pages of pdf document

In [ ]:
# importing required modules
import PyPDF2

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(stream=pdf_name)

# printing number of pages in pdf file
num_pages = pdfReader.numPages

In [ ]:
print(num_pages)

## Get tables

In [ ]:
import camelot

In [ ]:
# Syntax of the camelot.read_pdf function
tables = camelot.read_pdf(pdf_name, pages="1-{}".format(num_pages), flavor="stream")

## Format all tables

In [ ]:
i = 15
table_page1 = tables[i].df
table_page1

In [ ]:
deutsch_englisch = pd.DataFrame()

In [ ]:
for i in range(num_pages):
    # i = 14
    print(i)

    table_page1 = tables[i].df

    # remove first row
    table_page1 = table_page1[1:]

    if i == 0:
        # 1st row is column names
        table_page1 = table_page1[1:]

    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        # remove example
        del table_page1["beispiel"]
    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        table_page1["englisch"] = table_page1["englisch"].map(get_after_split)
        # remove example
        del table_page1["beispiel"]
    if i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14]:
        table_page1.columns = ["artikel", "deutsch_plural", "plural", "englisch"]
        # remove plural
        del table_page1["plural"]
    if i in [0]:
        table_page1.columns = [
            "artikel",
            "deutsch_plural",
            "plural",
            "englisch",
            "beispiel",
        ]
        # remove example & plural
        del table_page1["beispiel"]
        del table_page1["plural"]
    if i in []:
        table_page1.columns = ["artikel_deutsch", "plural", "englisch"]
        table_page1["artikel"], table_page1["deutsch_plural"] = zip(
            *table_page1["artikel_deutsch"].map(separate_artikel_deutsch)
        )
        del table_page1["artikel_deutsch"]
        del table_page1["plural"]
    if i in []:
        table_page1.columns = ["artikel_deutsch", "plural", "englisch", "beispiel"]
        table_page1["artikel"], table_page1["deutsch_plural"] = zip(
            *table_page1["artikel_deutsch"].map(separate_artikel_deutsch)
        )
        del table_page1["artikel_deutsch"]
        del table_page1["plural"]
        del table_page1["beispiel"]
    if i in [10]:
        table_page1.columns = ["artikel", "deutsch_plural", "plural_englisch"]
        table_page1["plural"], table_page1["englisch"] = zip(
            *table_page1["plural_englisch"].map(separate_plural_englisch)
        )
        del table_page1["plural_englisch"]
        del table_page1["plural"]

    # remove row when no word in deutsch and english
    table_page1 = table_page1[
        (table_page1["deutsch_plural"] != "") & (table_page1["englisch"] != "")
    ]

    # merge artikel and deutsch
    table_page1["deutsch_plural"] = table_page1[["artikel", "deutsch_plural"]].agg(
        " ".join, axis=1
    )

    # if article, add "the" to english
    table_page1["englisch"] = table_page1.apply(
        lambda x: ("the " + x["englisch"]) if (x["artikel"] != "") else x["englisch"],
        axis=1,
    )

    # remove artikel column
    del table_page1["artikel"]

    # if deutsch contains "(Sie", add "to" to englisch
    table_page1["is_verb"] = table_page1["deutsch_plural"].map(
        lambda x: ("(Sie" in x) or ("(ich" in x)
    )

    table_page1["englisch"] = table_page1.apply(
        lambda x: ("to " + x["englisch"])
        if (x["englisch"][:3] != "to ") & (x["is_verb"])
        else x["englisch"],
        axis=1,
    )

    # in deutsch or englisch, remove everything after the first "/" or "(" or "\n"
    table_page1["deutsch"] = table_page1["deutsch_plural"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )
    # in deutsch, remove everything after the first "/" or "(" or "\n"
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )

    # strip extra-space in word
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: re.sub("\s+", " ", x))
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.sub("\s+", " ", x)
    )

    # strip space on left and right
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: x.strip())
    table_page1["englisch"] = table_page1["englisch"].map(lambda x: x.strip())

    del table_page1["deutsch_plural"]
    del table_page1["is_verb"]

    # remove row when no word in deutsch or english
    table_page1 = table_page1[
        (table_page1["deutsch"] != "")
        & (table_page1["englisch"] != "")
        & (~table_page1["deutsch"].isna())
        & (~table_page1["englisch"].isna())
    ]

    deutsch_englisch = deutsch_englisch.append(table_page1[["deutsch", "englisch"]])

In [ ]:
deutsch_englisch

In [ ]:
deutsch_englisch.to_csv(
    "data/raw/new_vocabulary/wortschatzliste_einfach-gut_b1-1_englisch.csv", index=False
)

# wortschatzliste_einfach-gut_b1-2_englisch

In [ ]:
pdf_name = "data/raw/new_vocabulary/wortschatzliste_einfach-gut_b1-2_englisch.pdf"

## Get number of pages of pdf document

In [ ]:
# importing required modules
import PyPDF2

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(stream=pdf_name)

# printing number of pages in pdf file
num_pages = pdfReader.numPages

In [ ]:
print(num_pages)

## Get tables

In [ ]:
import camelot

In [ ]:
# Syntax of the camelot.read_pdf function
tables = camelot.read_pdf(pdf_name, pages="1-{}".format(num_pages), flavor="stream")

## Format all tables

In [ ]:
i = 0
table_page1 = tables[i].df
table_page1

In [ ]:
deutsch_englisch = pd.DataFrame()

In [ ]:
for i in range(num_pages):
    # i = 14
    print(i)

    table_page1 = tables[i].df

    # remove first row
    table_page1 = table_page1[1:]

    if i == 0:
        # 1st row is column names
        table_page1 = table_page1[1:]

    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        # remove example
        del table_page1["beispiel"]
    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        table_page1["englisch"] = table_page1["englisch"].map(get_after_split)
        # remove example
        del table_page1["beispiel"]
    if i in [1, 2, 4, 5, 6, 7, 8]:
        table_page1.columns = ["artikel", "deutsch_plural", "plural", "englisch"]
        # remove plural
        del table_page1["plural"]
    if i in []:
        table_page1.columns = [
            "artikel",
            "deutsch_plural",
            "plural",
            "englisch",
            "beispiel",
        ]
        # remove example & plural
        del table_page1["beispiel"]
        del table_page1["plural"]
    if i in [3, 9]:
        table_page1.columns = ["artikel_deutsch", "plural", "englisch"]
        table_page1["artikel"], table_page1["deutsch_plural"] = zip(
            *table_page1["artikel_deutsch"].map(separate_artikel_deutsch)
        )
        del table_page1["artikel_deutsch"]
        del table_page1["plural"]
    if i in []:
        table_page1.columns = ["artikel_deutsch", "plural", "englisch", "beispiel"]
        table_page1["artikel"], table_page1["deutsch_plural"] = zip(
            *table_page1["artikel_deutsch"].map(separate_artikel_deutsch)
        )
        del table_page1["artikel_deutsch"]
        del table_page1["plural"]
        del table_page1["beispiel"]
    if i in [10]:
        table_page1.columns = ["artikel", "deutsch_plural", "plural_englisch"]
        table_page1["plural"], table_page1["englisch"] = zip(
            *table_page1["plural_englisch"].map(separate_plural_englisch)
        )
        del table_page1["plural_englisch"]
        del table_page1["plural"]
    if i in [0]:
        table_page1.columns = [
            "artikel",
            "deutsch_plural",
            "plural_englisch",
            "beispiel",
        ]
        table_page1["plural"], table_page1["englisch"] = zip(
            *table_page1["plural_englisch"].map(separate_plural_englisch)
        )
        del table_page1["plural_englisch"]
        del table_page1["plural"]
        del table_page1["beispiel"]

    # remove row when no word in deutsch and english
    table_page1 = table_page1[
        (table_page1["deutsch_plural"] != "") & (table_page1["englisch"] != "")
    ]

    # merge artikel and deutsch
    table_page1["deutsch_plural"] = table_page1[["artikel", "deutsch_plural"]].agg(
        " ".join, axis=1
    )

    # if article, add "the" to english
    table_page1["englisch"] = table_page1.apply(
        lambda x: ("the " + x["englisch"]) if (x["artikel"] != "") else x["englisch"],
        axis=1,
    )

    # remove artikel column
    del table_page1["artikel"]

    # if deutsch contains "(Sie", add "to" to englisch
    table_page1["is_verb"] = table_page1["deutsch_plural"].map(
        lambda x: ("(Sie" in x) or ("(ich" in x)
    )

    table_page1["englisch"] = table_page1.apply(
        lambda x: ("to " + x["englisch"])
        if (x["englisch"][:3] != "to ") & (x["is_verb"])
        else x["englisch"],
        axis=1,
    )

    # in deutsch or englisch, remove everything after the first "/" or "(" or "\n"
    table_page1["deutsch"] = table_page1["deutsch_plural"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )
    # in deutsch, remove everything after the first "/" or "(" or "\n"
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )

    # strip extra-space in word
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: re.sub("\s+", " ", x))
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.sub("\s+", " ", x)
    )

    # strip space on left and right
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: x.strip())
    table_page1["englisch"] = table_page1["englisch"].map(lambda x: x.strip())

    del table_page1["deutsch_plural"]
    del table_page1["is_verb"]

    # remove row when no word in deutsch or english
    table_page1 = table_page1[
        (table_page1["deutsch"] != "")
        & (table_page1["englisch"] != "")
        & (~table_page1["deutsch"].isna())
        & (~table_page1["englisch"].isna())
    ]

    deutsch_englisch = deutsch_englisch.append(table_page1[["deutsch", "englisch"]])

In [ ]:
deutsch_englisch

In [ ]:
deutsch_englisch.to_csv(
    "data/raw/new_vocabulary/wortschatzliste_einfach-gut_b1-2_englisch.csv", index=False
)

# wortschatzliste_einfach-besser_b1-b2_englisch

In [ ]:
pdf_name = "data/raw/new_vocabulary/wortschatzliste_einfach-besser_b1-b2_englisch.pdf"

## Get number of pages of pdf document

In [ ]:
# importing required modules
import PyPDF2

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(stream=pdf_name)

# printing number of pages in pdf file
num_pages = pdfReader.numPages

In [ ]:
print(num_pages)

## Get tables

In [ ]:
import camelot

In [ ]:
# Syntax of the camelot.read_pdf function
tables = camelot.read_pdf(pdf_name, pages="1-{}".format(num_pages), flavor="stream")

## Format all tables

In [ ]:
i = 44
table_page1 = tables[i].df
table_page1

In [ ]:
deutsch_englisch = pd.DataFrame()

In [ ]:
for i in range(num_pages):
    # i = 14
    print(i)

    table_page1 = tables[i].df

    # remove first row
    table_page1 = table_page1[1:]

    if i == 0:
        # 1st row is column names
        table_page1 = table_page1[1:]

    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        # remove example
        del table_page1["beispiel"]
    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "englisch", "beispiel"]
        table_page1["englisch"] = table_page1["englisch"].map(get_after_split)
        # remove example
        del table_page1["beispiel"]
    if i in list(range(1, 44)):
        table_page1.columns = ["artikel", "deutsch_plural", "plural", "englisch"]
        # remove plural
        del table_page1["plural"]
    if i in [0]:
        table_page1.columns = [
            "artikel",
            "deutsch_plural",
            "plural",
            "englisch",
            "beispiel",
        ]
        # remove example & plural
        del table_page1["beispiel"]
        del table_page1["plural"]
    if i in []:
        table_page1.columns = ["artikel_deutsch", "plural", "englisch"]
        table_page1["artikel"], table_page1["deutsch_plural"] = zip(
            *table_page1["artikel_deutsch"].map(separate_artikel_deutsch)
        )
        del table_page1["artikel_deutsch"]
        del table_page1["plural"]
    if i in []:
        table_page1.columns = ["artikel_deutsch", "plural", "englisch", "beispiel"]
        table_page1["artikel"], table_page1["deutsch_plural"] = zip(
            *table_page1["artikel_deutsch"].map(separate_artikel_deutsch)
        )
        del table_page1["artikel_deutsch"]
        del table_page1["plural"]
        del table_page1["beispiel"]
    if i in []:
        table_page1.columns = ["artikel", "deutsch_plural", "plural_englisch"]
        table_page1["plural"], table_page1["englisch"] = zip(
            *table_page1["plural_englisch"].map(separate_plural_englisch)
        )
        del table_page1["plural_englisch"]
        del table_page1["plural"]
    if i in []:
        table_page1.columns = [
            "artikel",
            "deutsch_plural",
            "plural_englisch",
            "beispiel",
        ]
        table_page1["plural"], table_page1["englisch"] = zip(
            *table_page1["plural_englisch"].map(separate_plural_englisch)
        )
        del table_page1["plural_englisch"]
        del table_page1["plural"]
        del table_page1["beispiel"]

    # remove row when no word in deutsch and english
    table_page1 = table_page1[
        (table_page1["deutsch_plural"] != "") & (table_page1["englisch"] != "")
    ]

    # merge artikel and deutsch
    table_page1["deutsch_plural"] = table_page1[["artikel", "deutsch_plural"]].agg(
        " ".join, axis=1
    )

    # if article, add "the" to english
    table_page1["englisch"] = table_page1.apply(
        lambda x: ("the " + x["englisch"]) if (x["artikel"] != "") else x["englisch"],
        axis=1,
    )

    # remove artikel column
    del table_page1["artikel"]

    # if deutsch contains "(Sie", add "to" to englisch
    table_page1["is_verb"] = table_page1["deutsch_plural"].map(
        lambda x: ("(Sie" in x) or ("(ich" in x)
    )

    table_page1["englisch"] = table_page1.apply(
        lambda x: ("to " + x["englisch"])
        if (x["englisch"][:3] != "to ") & (x["is_verb"])
        else x["englisch"],
        axis=1,
    )

    # in deutsch or englisch, remove everything after the first "/" or "(" or "\n"
    table_page1["deutsch"] = table_page1["deutsch_plural"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )
    # in deutsch, remove everything after the first "/" or "(" or "\n"
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.split("\(|/|\n|,|-", x)[0]
    )

    # strip extra-space in word
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: re.sub("\s+", " ", x))
    table_page1["englisch"] = table_page1["englisch"].map(
        lambda x: re.sub("\s+", " ", x)
    )

    # strip space on left and right
    table_page1["deutsch"] = table_page1["deutsch"].map(lambda x: x.strip())
    table_page1["englisch"] = table_page1["englisch"].map(lambda x: x.strip())

    del table_page1["deutsch_plural"]
    del table_page1["is_verb"]

    # remove row when no word in deutsch or english
    table_page1 = table_page1[
        (table_page1["deutsch"] != "")
        & (table_page1["englisch"] != "")
        & (~table_page1["deutsch"].isna())
        & (~table_page1["englisch"].isna())
    ]

    deutsch_englisch = deutsch_englisch.append(table_page1[["deutsch", "englisch"]])

In [ ]:
deutsch_englisch

In [ ]:
deutsch_englisch.to_csv(
    "data/raw/new_vocabulary/wortschatzliste_einfach-besser_b1-b2_englisch.csv",
    index=False,
)